In [ ]:
import csv
from getpass import getpass
from time import sleep 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome

 #start the web driver
driver = Chrome()

def get_tweet_data(tweet):
    """extract data from tweet data"""
    username = tweet.find_element_by_xpath('.//span').text
    handle = tweet.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        postdate = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    respond = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + respond

    reply_count = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_count = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_number = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
   
    tweet_info = (username, handle, postdate, text, reply_count, retweet_count,like_number)
    return tweet_info
   
#navigate to twitter
driver.get('https://www.twitter.com/login')
driver.maximize_window()
#find username input

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
#send username info
username.send_keys('username or gmail')
my_password = '%myPassword'

#find password input
password =driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
 #Log in

password.send_keys(Keys.RETURN)

#get to search input
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('#request for startup')
search_input .send_keys(Keys.RETURN)

#go to latest tab
driver.find_element_by_link_text('Latest').click()
#get the tweets
tweet_data = []

tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")

scrolling = True

while scrolling:
       tweets = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
       for tweet in tweets[-15:]:
            data = get_tweet_data(tweet)
            if data:
                 tweet_id = ''.join(data)
                 if tweet_id not in tweet_ids:
                     tweet_ids.add(tweet_id)
                 tweet_data.append(data)
             
scroll_attempt = 0

while True:
    #Scrolling check the scroll state
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(3) #let the rpogram sleep :D connection is slow
    current_position = driver.execute_script("return window.pageYOffset;")

    if last_position == current_position:
        scroll_attempt += 1

        if scroll_attempt >= 5:
            scrolling = False
            break
        else:
            sleep(5) # try to scroll again
    else:
        last_position = current_position
        break

# close the web driver
driver.close()

In [5]:
#save data to the file
with open('tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)